In [1]:
from vertexai.preview.language_models import TextGenerationModel

In [91]:
model = TextGenerationModel.from_pretrained('text-bison')

In [18]:
# get project ID
P = ! gcloud config list --format 'value(core.project)'
PROJECT_ID = P[0]

In [ ]:
######################################################################################
#
# DEI dataset
#
######################################################################################

In [4]:
%%bigquery dei_df --project {PROJECT_ID}

SELECT * FROM `bigquery-public-data.google_dei.dar_intersectional_representation` LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
dei_df.head(5)

,workforce,report_year,gender_us,race_asian,race_black,race_hispanic_latinx,race_native_american,race_white
0,tech,2018,men,0.310,0.015,0.036,0.005,0.443
1,tech,2023,men,0.347,0.029,0.049,0.005,0.334
2,tech,2020,men,0.335,0.019,0.038,0.005,0.389
3,tech,2019,men,0.321,0.016,0.037,0.005,0.417
4,tech,2015,men,0.275,0.013,0.033,0.005,0.515


In [ ]:
######################################################################################
# generate text from the tabular data
######################################################################################

In [9]:
# Limit the size of the table due to context window
dei_table = dei_df.head(5)

prompt = f"""
  Using the following table that shows the percentage of men in the tech workforce in the 
  United States by race and ethnicity, generate a sentence for each row to describe the meaning of the values

  {dei_table}

"""
response = model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8)
next_input = response.text
response

In 2018, 31% of men in the tech workforce in the United States were Asian.
In 2018, 0.015% of men in the tech workforce in the United States were black.
In 2018, 0.036% of men in the tech workforce in the United States were Hispanic or Latino.
In 2018, 0.005% of men in the tech workforce in the United States were Native American.
In 2018, 44.3% of men in the tech workforce in the United States were white.

In 2023, 34.7% of men in the tech workforce in the United States were Asian.
In 2023, 0.029% of men in the tech workforce in the United States were black.
In 2023, 0.049% of men in the tech workforce in the United States were Hispanic or Latino.
In 2023, 0.005% of men in the tech workforce in the United States were Native American.
In 2023, 33.4% of men in the tech workforce in the United States were white.

In 2020, 33.5% of men in the tech workforce in the United States were Asian.
In 2020, 0.019% of men in the tech workforce in the United States were black.
In 2020, 0.038% of men 

In [ ]:
######################################################################################
# use the generated text to create summaries
######################################################################################

In [16]:
prompt = f"""
  Using the following sentences to generate a paragraph summary of the trends that are occurring

  {next_input}
  
  Do not add any insights or conclusions that are not found in the data provided.  
  Identify the main trends in growth or decline over time.
  Compare and contrast the trends in different groups.

"""
response = model.predict(prompt, temperature=0.1, max_output_tokens=1024, top_k=20, top_p=0.4)
response

The data shows that the percentage of men in the tech workforce in the United States who are Asian has increased from 27.5% in 2015 to 34.7% in 2023. The percentage of men in the tech workforce who are black has remained relatively unchanged at 0.015% from 2015 to 2023. The percentage of men in the tech workforce who are Hispanic or Latino has increased from 0.033% in 2015 to 0.049% in 2023. The percentage of men in the tech workforce who are Native American has remained at 0.005% from 2015 to 2023. The percentage of men in the tech workforce who are white has decreased from 51.5% in 2015 to 33.4% in 2023.

The data also shows that the percentage of men in the tech workforce in the United States who are Asian is higher than the percentage of men in the tech workforce who are black, Hispanic or Latino, or Native American. The percentage of men in the tech workforce who are white is lower than the percentage of men in the tech workforce who are Asian, black, Hispanic or Latino, or Native